In [4]:
import findspark
#findspark.init('/usr/hdp/2.6.1.0-129/spark2')
findspark.init()
import pyspark
import numpy as np
import pandas as pd
from pyspark.sql import SparkSession

In [5]:
from pyspark import SparkContext
sc=SparkContext()

In [6]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [7]:
s_df = sqlContext.read.csv("C:/big_data_log/big_data-2019-02-*.log", sep=';').cache()

In [8]:
s_products = sqlContext.read.format('jdbc').options(
          url='jdbc:mysql://10.1.3.244/tm_datasc',
          driver='com.mysql.jdbc.Driver',
          dbtable='products',
          user='datasc',
          password='scdata',
          serverTimezone='GMT',
          zeroDateTimeBehavior='CONVERT_TO_NULL').load()

categories = sqlContext.read.format('jdbc').options(
          url='jdbc:mysql://10.1.3.244/tm_datasc',
          driver='com.mysql.jdbc.Driver',
          dbtable='categories',
          user='datasc',
          password='scdata',
          serverTimezone='GMT',
          zeroDateTimeBehavior='CONVERT_TO_NULL').load() 
          
products_categories = sqlContext.read.format('jdbc').options(
          url='jdbc:mysql://10.1.3.244/tm_datasc',
          driver='com.mysql.jdbc.Driver',
          dbtable='products_categories',
          user='datasc',
          password='scdata',
          serverTimezone='GMT',
          zeroDateTimeBehavior='CONVERT_TO_NULL').load()
          
categories_content = sqlContext.read.format('jdbc').options(
          url='jdbc:mysql://10.1.3.244/tm_datasc',
          driver='com.mysql.jdbc.Driver',
          dbtable='categories_content',
          user='datasc',
          password='scdata',
          serverTimezone='GMT',
          zeroDateTimeBehavior='CONVERT_TO_NULL').load()
          
products_content = sqlContext.read.format('jdbc').options(
          url='jdbc:mysql://10.1.3.244/tm_datasc',
          driver='com.mysql.jdbc.Driver',
          dbtable='products_content',
          user='datasc',
          password='scdata',
          serverTimezone='GMT',
          zeroDateTimeBehavior='CONVERT_TO_NULL').load()

In [44]:
orders = sqlContext.read.format('jdbc').options(
          url='jdbc:mysql://10.1.3.244/tm_datasc',
          driver='com.mysql.jdbc.Driver',
          dbtable='orders',
          user='datasc',
          password='scdata',
          serverTimezone='GMT',
          zeroDateTimeBehavior='CONVERT_TO_NULL').load()

order_items = sqlContext.read.format('jdbc').options(
          url='jdbc:mysql://10.1.3.244/tm_datasc',
          driver='com.mysql.jdbc.Driver',
          dbtable='order_items',
          user='datasc',
          password='scdata',
          serverTimezone='GMT',
          zeroDateTimeBehavior='CONVERT_TO_NULL').load()

In [9]:
s_df = s_df.selectExpr('_c0 as data_id', 
                   '_c1 as data_type', 
                   '_c2 as view_type',
                   '_c3 as customer_id',
                   '_c4 as vote_index',
                   '_c5 as added_qty',
                   '_c6 as token',
                   '_c7 as log_time',
                   '_c8 as session_id',
                   '_c9 as ip_address',
                   '_c10 as device_type',
                   '_c11 as click_success')

s_df = s_df.select(s_df.data_id.cast("integer"),
               s_df.view_type.cast("integer"),
               s_df.data_type.cast("integer"),
               s_df.customer_id.cast("integer"),
               s_df.added_qty.cast("integer"),
               s_df.token.cast("string"),
               s_df.log_time.cast("string"),
               s_df.session_id.cast("string"),
               s_df.ip_address.cast("string"),
               s_df.device_type.cast("integer"),
               s_df.click_success.cast("integer"))


s_df = s_df.fillna({'click_success':'0'})
s_df = s_df.fillna({'added_qty':'0'})
s_df = s_df.fillna({'data_id':'0'})
s_df = s_df.fillna({'view_type':'0'})
s_df = s_df.fillna({'data_type':'0'})

s_df = s_df.filter(s_df.click_success<3)
s_df = s_df.filter(s_df.added_qty<100)
s_df = s_df.filter(s_df.view_type<25)
s_df = s_df.filter(s_df.data_type<25)
s_df = s_df.filter(s_df.device_type<2)
s_df = s_df.filter(s_df.click_success<3)

In [10]:
from pyspark.sql.types import IntegerType, TimestampType
from pyspark.sql.functions import unix_timestamp, from_unixtime

s_df = s_df.withColumn('log_time', unix_timestamp("log_time", "yyyy-MM-dd HH:mm:ss").cast('timestamp').alias("log_time"))
s_df = s_df.withColumnRenamed('customer_cookie_token', 'token')
s_df = s_df.drop('id')
s_df = s_df.cache()

In [11]:
from pyspark.sql import functions as F
from pyspark.sql.functions import *
from pyspark.sql.functions import col, size
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf
from pyspark.sql.functions import datediff, to_date, lit

break_point = '2017-04-29 00:00:00'

s_df = s_df.filter(s_df.log_time>break_point)
s_df = s_df.withColumn('date', s_df['log_time'].cast('date'))
s_df = s_df.withColumn('prod', F.when(s_df.data_type == 1, s_df.data_id).otherwise(0))
s_df = s_df.withColumn('cat', F.when(s_df.data_type == 7, s_df.data_id).otherwise(0))
s_df = s_df.withColumn('class', F.when(s_df.click_success == 1, s_df.click_success).otherwise(0))

In [12]:
s_df = s_df.dropDuplicates()

In [13]:
from pyspark.sql.functions import date_format

s_df = s_df.withColumn('year', year("log_time").alias('year'))
s_df = s_df.withColumn('month', month("log_time").alias('month'))
s_df = s_df.withColumn('wday', date_format('date', 'u'))
s_df = s_df.withColumn('hour', hour("log_time").alias('hour'))
s_df = s_df.withColumn('minute', minute("log_time").alias('minute'))
s_df = s_df.select(s_df.data_id,
               s_df.view_type,
               s_df.data_type,
               s_df.customer_id,
               s_df.added_qty,
               s_df.token,
               s_df.log_time,
               s_df.session_id,
               s_df.ip_address,
               s_df.device_type,
               s_df.click_success,
               s_df.date,
               s_df.prod.cast("integer"),
               s_df.cat.cast("integer"),
               s_df.year.cast("integer"),
               s_df.month.cast("integer"),
               s_df.wday.cast("integer"),
               s_df.hour.cast("integer"),
               s_df.minute.cast("integer"))
s_df = s_df.cache()

In [14]:
s_df = s_df.filter("token is not NULL")
s_df = s_df.filter("session_id is not NULL")
s_df = s_df.filter("log_time is not NULL")
s_df = s_df.filter("data_type is not NULL")
s_df = s_df.filter("view_type is not NULL")
s_df = s_df.filter("added_qty is not NULL")
s_df = s_df.filter("year is not NULL")
s_df = s_df.filter("month is not NULL")
s_df = s_df.filter("wday is not NULL")
s_df = s_df.filter("hour is not NULL")
s_df = s_df.filter("minute is not NULL")
s_df = s_df.filter("prod is not NULL")

In [15]:
categories = categories_content.select('category_id', 'name')
categories = categories.withColumnRenamed("name", "category_name")

products_content = products_content.select('product_id', 'name', 'type')
products_content = products_content.withColumnRenamed("name", "product_name")

s_products = s_products.select('product_id', 'brand', 'cena_niza_redovna', 'status')
s_products = s_products.withColumnRenamed("cena_niza_redovna", "price")
s_products = s_products.filter('status==1')

In [16]:
s_products = s_products.alias('a').join(products_content.alias('b'), s_products['product_id']==products_content['product_id'], 'inner').select('a.product_id', 'b.product_name', 'b.type', 'a.brand', 'a.price')

s_products = s_products.alias('a').join(products_categories.alias('b'), s_products['product_id']==products_categories['product_id'], 'inner').select('a.product_id', 'a.product_name', 'a.type', 'b.category_id', 'a.brand', 'a.price')

s_products = s_products.alias('a').join(categories.alias('b'), s_products['category_id']==categories['category_id'], 'inner').select('a.product_id', 'a.product_name', 'a.type', 'b.category_id', 'b.category_name','a.brand', 'a.price')

In [17]:
from pyspark.sql.functions import monotonically_increasing_id

tokens = s_df.select('token').distinct()
tokens = tokens.withColumn("user", monotonically_increasing_id())

tokens = tokens.cache()

s_df = s_df.alias('a').join(tokens.alias('b'), s_df['token'] == tokens['token'], 'left')\
.select('a.data_id', 'a.view_type', 'a.data_type', 'a.customer_id', 'a.added_qty', 'a.token', 'b.user', 'a.log_time', 'a.ip_address', 'a.session_id', 'a.device_type', 'a.click_success', 'a.prod', 'a.cat', 'a.date','a.year', 'a.month', 'a.wday', 'a.hour', 'a.minute')\
.cache()

In [18]:
s_df.count()

1670512

In [19]:
s_df.show()

+-------+---------+---------+-----------+---------+--------------------+----+-------------------+---------------+--------------------+-----------+-------------+------+---+----------+----+-----+----+----+------+
|data_id|view_type|data_type|customer_id|added_qty|               token|user|           log_time|     ip_address|          session_id|device_type|click_success|  prod|cat|      date|year|month|wday|hour|minute|
+-------+---------+---------+-----------+---------+--------------------+----+-------------------+---------------+--------------------+-----------+-------------+------+---+----------+----+-----+----+----+------+
|  75936|        7|        1|          0|        0|000344tlgncv5k5lr...|1059|2019-02-03 19:59:03|   66.249.92.33|000344tlgncv5k5lr...|          0|            0| 75936|  0|2019-02-03|2019|    2|   7|  19|    59|
| 128207|        7|        1|          0|        0|00750610f0d326cb6...| 235|2019-02-08 18:07:20|  93.87.106.241|pvt567f3l2oaadded...|          0|          

In [20]:
s_df.printSchema()

root
 |-- data_id: integer (nullable = true)
 |-- view_type: integer (nullable = true)
 |-- data_type: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- added_qty: integer (nullable = true)
 |-- token: string (nullable = true)
 |-- user: long (nullable = true)
 |-- log_time: timestamp (nullable = true)
 |-- ip_address: string (nullable = true)
 |-- session_id: string (nullable = true)
 |-- device_type: integer (nullable = true)
 |-- click_success: integer (nullable = true)
 |-- prod: integer (nullable = true)
 |-- cat: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- wday: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- minute: integer (nullable = true)



In [21]:
s_df = s_df.filter("customer_id>0")
s_df.count()

7624

In [22]:
s_df = s_df.withColumn("current_date", F.date_format(F.current_date(), "y-M-d")).withColumn("log_date", F.date_format(s_df.date, "y-M-d"))
s_df = s_df.withColumn('current_date', s_df['current_date'].cast('date'))
s_df = s_df.withColumn('log_date', s_df['log_date'].cast('date'))
s_df = s_df.withColumn("days_between", F.datediff(s_df.current_date, s_df.date))

In [23]:

s_df.printSchema()

root
 |-- data_id: integer (nullable = true)
 |-- view_type: integer (nullable = true)
 |-- data_type: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- added_qty: integer (nullable = true)
 |-- token: string (nullable = true)
 |-- user: long (nullable = true)
 |-- log_time: timestamp (nullable = true)
 |-- ip_address: string (nullable = true)
 |-- session_id: string (nullable = true)
 |-- device_type: integer (nullable = true)
 |-- click_success: integer (nullable = true)
 |-- prod: integer (nullable = true)
 |-- cat: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- wday: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- minute: integer (nullable = true)
 |-- current_date: date (nullable = true)
 |-- log_date: date (nullable = true)
 |-- days_between: integer (nullable = true)



In [24]:
s_df.select("customer_id", "log_date", "days_between").show()

+-----------+----------+------------+
|customer_id|  log_date|days_between|
+-----------+----------+------------+
|      32477|2019-02-04|           8|
|     544369|2019-02-02|          10|
|     544369|2019-02-02|          10|
|     544369|2019-02-02|          10|
|     544369|2019-02-02|          10|
|     544369|2019-02-02|          10|
|     544369|2019-02-02|          10|
|     544369|2019-02-02|          10|
|     208691|2019-02-10|           2|
|     208691|2019-02-10|           2|
|     156367|2019-02-05|           7|
|     444110|2019-02-06|           6|
|     444110|2019-02-06|           6|
|     444110|2019-02-06|           6|
|     444110|2019-02-06|           6|
|     528559|2019-02-06|           6|
|     528559|2019-02-06|           6|
|     528559|2019-02-06|           6|
|     528559|2019-02-06|           6|
|     528559|2019-02-06|           6|
+-----------+----------+------------+
only showing top 20 rows



In [25]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, col

window = Window.partitionBy(s_df['customer_id']).orderBy(s_df['log_date'].asc())

In [26]:
n=1
first_transaction = s_df.select(col('customer_id'), col('log_date'), row_number().over(window).alias('row_number')) \
  .where(col('row_number') <= n).drop('row_number')

In [27]:
first_transaction = first_transaction.withColumnRenamed('log_date', 'first_transaction')

In [28]:
window = Window.partitionBy(s_df['customer_id']).orderBy(s_df['log_date'].desc())

In [29]:
last_transaction = s_df.select(col('customer_id'), col('log_date'), row_number().over(window).alias('row_number')) \
  .where(col('row_number') <= n).drop('row_number')

In [30]:
last_transaction = last_transaction.withColumnRenamed('log_date', 'last_transaction')

In [31]:
#shopping_frequency (num of orders per customer_id)
frequency = s_df.groupBy("customer_id", "data_type").count().filter("data_type==11").orderBy("count", ascending=False).drop('data_type')

In [32]:
#recency (difference between last transaction and today)
recency = last_transaction.withColumn("current_date", F.date_format(F.current_date(), "y-M-d")).withColumn("log_date", F.date_format(last_transaction.last_transaction, "y-M-d"))
recency = recency.withColumn("days_between", F.datediff(recency.current_date, recency.last_transaction))
recency = recency.withColumnRenamed('days_between','recency').drop('last_transaction', 'current_date')

In [33]:
length = first_transaction.join(last_transaction, ['customer_id'], 'inner')

In [34]:
length = length.withColumn("length", F.datediff(length.last_transaction, length.first_transaction))

In [35]:
length.show(100)

+-----------+-----------------+----------------+------+
|customer_id|first_transaction|last_transaction|length|
+-----------+-----------------+----------------+------+
|      94819|       2019-02-02|      2019-02-02|     0|
|     413073|       2019-01-31|      2019-01-31|     0|
|     459999|       2019-02-07|      2019-02-07|     0|
|     493490|       2019-02-07|      2019-02-07|     0|
|     547213|       2019-02-08|      2019-02-09|     1|
|     105437|       2019-01-31|      2019-02-08|     8|
|     444553|       2019-01-31|      2019-01-31|     0|
|     544177|       2019-01-31|      2019-01-31|     0|
|     545725|       2019-02-05|      2019-02-05|     0|
|     545792|       2019-02-05|      2019-02-05|     0|
|     546883|       2019-02-07|      2019-02-07|     0|
|     502742|       2019-02-01|      2019-02-07|     6|
|     544777|       2019-02-01|      2019-02-02|     1|
|     546284|       2019-02-06|      2019-02-06|     0|
|     546829|       2019-02-07|      2019-02-08|

In [36]:
length.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- first_transaction: date (nullable = true)
 |-- last_transaction: date (nullable = true)
 |-- length: integer (nullable = true)



In [37]:
recency.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- log_date: string (nullable = true)
 |-- recency: integer (nullable = true)



In [38]:
#fali monetary i frequency

In [39]:
s_df.printSchema()

root
 |-- data_id: integer (nullable = true)
 |-- view_type: integer (nullable = true)
 |-- data_type: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- added_qty: integer (nullable = true)
 |-- token: string (nullable = true)
 |-- user: long (nullable = true)
 |-- log_time: timestamp (nullable = true)
 |-- ip_address: string (nullable = true)
 |-- session_id: string (nullable = true)
 |-- device_type: integer (nullable = true)
 |-- click_success: integer (nullable = true)
 |-- prod: integer (nullable = true)
 |-- cat: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- wday: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- minute: integer (nullable = true)
 |-- current_date: date (nullable = true)
 |-- log_date: date (nullable = true)
 |-- days_between: integer (nullable = true)



In [40]:
frequency = s_df.groupBy("customer_id").count().withColumnRenamed("count", "frequency")

In [41]:
frequency.show()

+-----------+---------+
|customer_id|frequency|
+-----------+---------+
|     547213|       19|
|     493490|        3|
|     459999|        3|
|      94819|        2|
|     413073|        3|
|     544177|        4|
|     444553|        2|
|     545725|        6|
|     105437|        6|
|     546883|        8|
|     545792|       14|
|     546829|        4|
|     544777|       47|
|     502742|        3|
|     547827|        5|
|     546284|        2|
|     545983|        2|
|     545186|        8|
|     543014|        1|
|     544231|        3|
+-----------+---------+
only showing top 20 rows



In [61]:
monetary = s_df.filter("data_type==11")

In [62]:
orders = orders.select("order_id", "customer_id", "total_value")

In [63]:
monetary = monetary.join(orders, monetary['data_id']==orders['order_id'], 'inner').select(monetary['customer_id'], 'total_value')

In [64]:
monetary = monetary.groupBy("customer_id").sum()

In [66]:
monetary = monetary.drop("sum(customer_id)").withColumnRenamed("sum(total_value)", "monetary")

In [67]:
monetary.show()

+-----------+--------+
|customer_id|monetary|
+-----------+--------+
|     413073|   29479|
|     459999|   29499|
|      94819|   33051|
|     444553|    2580|
|     545792|   34290|
|     105437|   21591|
|     546284|    5990|
|     545983|    7999|
|     547143|    1788|
|     544035|    5390|
|     547683|   47898|
|     544231|    3861|
|     547645|   37399|
|     519133|   49090|
|     545546|    3699|
|     544347|   35991|
|     541850|   65980|
|     546758|   24439|
|     545024|   25990|
|     546969|    2029|
+-----------+--------+
only showing top 20 rows



In [69]:
print("Monetary: ",monetary.count())
print("Frequency: ",frequency.count())
print("Length: ",length.count())
print("Recency: ",recency.count())

Monetary:  614
Frequency:  1224
Length:  1224
Recency:  1224


In [71]:
lrfm = frequency.join(length, ['customer_id'], 'inner').join(recency, ['customer_id'], 'inner').join(monetary, ['customer_id'], 'left').fillna(0)

In [73]:
lrfm = lrfm.drop("first_transaction", "last_transaction", "log_date")

In [ ]:
lrfm.show()